<img src="escudo_utfsm.gif" style="float:right;height:100px">
<img src="IsotipoDIisocolor.png" style="float:left;height:100px">
<center>
    <h1> ILI285 - Computación Científica I / INF285 - Computación Científica</h1>
    <h1> Tarea 3: Interpolación  </h1>
    <h3> [S]cientific [C]omputing [T]eam 2019</h3>
</center>
<p>
<center>Junio 2019 - v2.0 </center>
</p>

---

## Contexto

Hasta hace no más de una década, los dibujos animados y videojuegos eran visualizados con un *framerate* de entre 12 a 24 *frames* por segundo (*FPS*). Sin embargo, las mejoras en las tecnologías nos permiten hoy disfrutar de videos con una mucho mejor fluidez, siendo el *framerate* estándar actual de 60 *FPS*.

### ¿Qué es Interpolación de Movimiento o *Motion-Compensated Frame Interpolation* (MCFI)? 

Corresponde a un método de **procesamiento de video** en el cual se generan *frames* de animación entre los *frames* existentes, utilizando métodos de interpolación. El objetivo principal es mejorar la fluidez del video.

En esta tarea implementaremos MCFI aplicando los algoritmos de interpolación aprendidos en la clase de Computación Científica, para mejorar el *framerate* de un video que posee $N$ FPS (*Frames Per Second*), para luego generar un nuevo video con un total de *frames* mayor al original.

---

#### Librerías

In [3]:
from __future__ import print_function
import numpy as np
#pip3 install imutils
import imutils
#pip3 install opencv-python
import cv2 
import matplotlib.pyplot as plt
import scipy.interpolate as scp
import base64
from IPython.display import HTML, clear_output

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import time

---

## Problema

El problema, descrito de forma general, corresponde al incremento de los FPS de un video, esperando mejorar la fluidez de este. Junto a esta tarea, se adjuntan cuatro videos. El nombre de cada video sigue el formato `video_Nfps.mp4` [1], donde $N$ indica la cantidad de FPS del video. Para esta tarea, trabajaremos con videos con 60, 30, 15 y 5 FPS. Una vez incrementados los FPS, compararemos los resultados con la versión *original* del video.

Para ilustrar de manera inicial lo que se desea realizar, considere un video compuesto por los 2 siguientes *frames*:

<img src="interpolacion2.png" style="float:center;height:200px">

Su algoritmo de interpolación debe ser capaz de construir una o más funciones, utilizando como *data* los frames disponibles del video. En este ejemplo, podríamos construir una interpolación utilizando como datos los dos *frames* disponibles y, mediante esta función, se pueden generar uno o más *frames* dentro de la secuencia para aumentar el *framerate* de su video:

<img src="interpolacion3.png" style="float:center;height:198px">

---

## Sección 1 (40 puntos). Testing de métodos de interpolación

En esta pregunta implementaremos una forma sencilla de interpolación de videos. Considere que el video contiene un total de $M$ frames.

El proceso de interpolación será **a través de cada pixel**, es decir, si se desea interpolar el pixel $(i, j)$ y generar un nuevo frame, entonces el interpolador se debe construir utilizando como *data* todos los pixeles $(i, j)$ del video. De una forma más matemática, el conjunto de datos está definido por:

$$
    S_{i,j} = \{ (x_1, I_1[i, j]), (x_2, I_2[i,j]), \dots, (x_M, I_M[i,j]) \},
$$

donde $x_i$ corresponde a un punto siguiendo una distribución equiespaciada o de Chebyshev e $I_k[i,j]$ denota el valor del pixel $(i,j)$ del $k$-ésimo frame. Su labor en esta pregunta es interpolar los datos de $S_{i,j}$ y así construir los pixeles que se desean agregar entre cada par de frames.

1. Implemente la función _interpolate_\__frames (frames, dst_\__fps, seconds, interpolator)_. Esta función recibe como parámetros:
    * La matriz de _frames_: Video original a interpolar.
    * La cantidad original de *fps*.
    * La cantidad de _fps_ deseados (mayor a los _fps_ originales).
    * La cantidad de segundos que tendrá el video.
    * Un string _interpolator_, cuyos valores posibles son:
        * "spline-cubica": Spline cúbica. Estudie y seleccione un tipo de condición de borde adecuado para el problema, justificando su elección.
        * "spline-lineal": Spline lineal.
        * "lagrange": Interpolación polinomial mediante el método Interpolación de Lagrange.
        * "baricentrica": Interpolación Baricéntrica [2].
    * Un string 'points_type', cuyos valores posibles son:
        * "equi": Para usar una distribución equiespaciada de puntos en el eje de las abscisas.
        * "cheb": Los x_i (o eje de las abscisas) son los puntos de Chebyshev.

    En caso de utilizar código ajeno, ya sean librerías, notebooks del curso u otro, recuerde referenciar apropiadamente.<br/><br/>

2. Mediante la función implementada anteriormente, aumentaremos los *framerates* de los videos disponibles. Para esto:
    * Tomando como base el _video_\__30fps.mp4_, genere un video de 60 fps.
    * Tomando como base el _video_\__15fps.mp4_, genere un video de 30 fps.
    * Tomando como base el _video_\__5fps.mp4_, genere un video de 15 fps.
    * Tomando como base el _video_\__5fps.mp4_, genere un video de 60 fps.

    Para esto, utilize los 4 interpoladores generados y las distribuciones de puntos especificadas. Mida los tiempos de cómputos de cada método, con cada distribución de puntos.

In [8]:
'''
Carga de video a matriz de [height, width, fps*seconds], dejando en escala de grises cada frame, 
reduciendo su dimensionalidad.

Input:
    src_name: nombre (o path) de video de origen
    seconds: cantidad de tiempo en segundos a considerar del video de origen.

Output: 
    frames: matriz de shape [height, width, fps*seconds], la cual contiene los frames del video original
    number_fps: frames por segundo del video original
'''
def video_to_matrix(src_name, seconds):
    # Se lee el video
    stream = cv2.VideoCapture(src_name)
    
    # Dimension del video y fps
    height = round(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = round(stream.get(cv2.CAP_PROP_FRAME_WIDTH))
    number_fps = stream.get(cv2.CAP_PROP_FPS)

    frames = np.zeros((height, width, round(number_fps)*seconds))

    for i in range(round(number_fps)*seconds):
        # Leer el video hasta que no hayan más datos
        (grabbed, frame) = stream.read()
        if not grabbed:
            break
            
        # Leer el frame y cambiarlo a escala de grises
        frame = cv2.resize(frame, (width, height)) 
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames[:,:,i] = frame
        
    return frames

In [9]:
# 
def matrix_to_video(filename, video_matrix, framerate, width, height):
    out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc('m','p','4','v'), framerate, (width,height))
   
    for i in range(video_matrix.shape[2]):
        frame = np.uint8(video_matrix[:,:,i])
        out.write(np.dstack([frame, frame, frame]))
    
    out.release()
    cv2.destroyAllWindows() 
    return "Done"


In [10]:
def cubic_spline_interpolator(old_xs, old_ys, new_xs):
    # https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.interpolate.CubicSpline.html
    aux_interpolator = scp.CubicSpline(old_xs, old_ys)
    return aux_interpolator(new_xs)

def linear_spline_interpolator(old_xs, old_ys, new_xs):
    def aux_interpolator(x):
        for i in range(len(old_xs) - 1):
            if old_xs[i] <= x and x <= old_xs[i+1]:
                x_0, x_1 = float(old_xs[i]), float(old_xs[i+1])
                y_0, y_1 = float(old_ys[i]), float(old_ys[i+1])
                
                m = (y_1 - y_0) / (x_1 - x_0)
                y = m*x - m*x_0 + y_0
                return y
        return 0

    new_y = np.zeros(len(new_xs))
    for i in range(len(new_xs)):
        new_y[i] = aux_interpolator(new_xs[i])
    return new_y

def lagrange_interpolator(old_xs, old_ys, new_xs):
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.lagrange.html
    aux_interpolator = scp.lagrange(old_xs, old_ys)
    return aux_interpolator(new_xs)

def baricentrica_interpolator(old_xs, old_ys, new_xs):
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.BarycentricInterpolator.html
    aux_interpolator = scp.BarycentricInterpolator(old_xs, old_ys)
    return aux_interpolator(new_xs)

def chebyshev(i, n):
    assert 1 <= i and i <= n
    return np.cos((2*i-1)*np.pi/(2*n))

In [11]:
'''
frames: array of frames (height x width x total_frames)
old_fps: Number of fps of the original video
new_fps: fps of output matrix 
seconds: seconds of video 
interpolator: Interpolator type
points_type: Points distribution to use

return: dst_frames (array of frames (height x width x dst_fps*seconds))
'''
def interpolate_frames(frames, old_fps, new_fps, seconds, interpolator, points_type):
    interpolator_functions = {"spline-cubica": cubic_spline_interpolator, 
                              "spline-lineal": linear_spline_interpolator, 
                              "lagrange": lagrange_interpolator, 
                              "baricentrica": baricentrica_interpolator}
    height = len(frames)
    width = len(frames[0])

    dst_frames = np.zeros((height, width, round(new_fps)*seconds))
    
    if "cheb" == points_type:
        old_n = old_fps*seconds
        nombers_old = np.arange(0+1, old_n+1)
        old_x = np.zeros(old_n)
        for i in nombers_old:
            old_x[old_n-i] = chebyshev(i, old_n)
        # print(old_x)

        new_n = new_fps*seconds
        nombers_new = np.arange(0+1, new_n+1)
        new_x = np.zeros(new_n)
        for i in nombers_new:
            new_x[new_n-i] = chebyshev(i, new_n)

    elif points_type == "equi":
        old_x = np.arange(0, new_fps*seconds, new_fps/old_fps)
        new_x = np.arange(0, new_fps*seconds)
    else:
        return "ERROR"
    
    first_time = time.perf_counter()
    lapso_time = first_time
    inter_func = interpolator_functions[interpolator]
    for i in range(height):
        clear_output()
        print(i,"/",height)
        actual_time = time.perf_counter()
        print("\tTranscurrido total:",actual_time-first_time,"s")
        print("\tTranscurrido lapso:",actual_time-lapso_time,"s")
        print("\tRestante:",(actual_time-lapso_time)*(height-i),"s")
        lapso_time = actual_time
        print()
        for j in range(width):
            # old_y = np.array(map(float, frames[i][j][:len(old_x)]))
            old_y = frames[i][j][:len(old_x)]
            dst_frames[i][j] = inter_func(old_x, old_y, new_x)

    return dst_frames

In [25]:
def interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos_type, seconds = 5):
    matriz_de_video = video_to_matrix("video_"+str(old_fps)+"fps.mp4", seconds)
    matriz_interpolada = interpolate_frames(matriz_de_video, old_fps, dst_fps, seconds, interpolador, puntos_type)

    width, height = matriz_interpolada.shape[1], matriz_interpolada.shape[0]
    print(nombre)
    resultado = matrix_to_video(nombre, matriz_interpolada, dst_fps, width, height)
    return resultado


Spline-cubica

In [26]:
interpolador = "spline-cubica"
old_fps = 30
dst_fps = 60
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 51.29638153199994 s
	Transcurrido lapso: 0.14177434899966102 s
	Restante: 0.14177434899966102 s

30to60_spline-cubica_equi.mp4


'Done'

In [28]:
interpolador = "spline-cubica"
old_fps = 30
dst_fps = 60
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 50.50482717900013 s
	Transcurrido lapso: 0.14384160300005533 s
	Restante: 0.14384160300005533 s

30to60_spline-cubica_cheb.mp4


'Done'

In [29]:
interpolador = "spline-cubica"
old_fps = 15
dst_fps = 30
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 47.74824547699973 s
	Transcurrido lapso: 0.13287439699979586 s
	Restante: 0.13287439699979586 s

15to30_spline-cubica_equi.mp4


'Done'

In [30]:
interpolador = "spline-cubica"
old_fps = 15
dst_fps = 30
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 47.69533963599997 s
	Transcurrido lapso: 0.1319870830002401 s
	Restante: 0.1319870830002401 s

15to30_spline-cubica_cheb.mp4


'Done'

In [31]:
interpolador = "spline-cubica"
old_fps = 5
dst_fps = 15
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 45.6527372139999 s
	Transcurrido lapso: 0.12506640699984928 s
	Restante: 0.12506640699984928 s

5to15_spline-cubica_equi.mp4


'Done'

In [32]:
interpolador = "spline-cubica"
old_fps = 5
dst_fps = 15
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 46.15971849100015 s
	Transcurrido lapso: 0.13146149000021978 s
	Restante: 0.13146149000021978 s

5to15_spline-cubica_cheb.mp4


'Done'

In [33]:
interpolador = "spline-cubica"
old_fps = 5
dst_fps = 60
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 49.892873581 s
	Transcurrido lapso: 0.13645084100016902 s
	Restante: 0.13645084100016902 s

5to60_spline-cubica_equi.mp4


'Done'

In [35]:
interpolador = "spline-cubica"
old_fps = 5
dst_fps = 60
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 47.53604554499998 s
	Transcurrido lapso: 0.12148241200020493 s
	Restante: 0.12148241200020493 s

5to60_spline-cubica_cheb.mp4


'Done'

spline-lineal

In [36]:
interpolador = "spline-lineal"
old_fps = 30
dst_fps = 60
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 12984.252803819 s
	Transcurrido lapso: 36.52935028600041 s
	Restante: 36.52935028600041 s

30to60_spline-lineal_equi.mp4


'Done'

In [37]:
interpolador = "spline-lineal"
old_fps = 30
dst_fps = 60
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 2368.2602621690003 s
	Transcurrido lapso: 6.763097611001285 s
	Restante: 6.763097611001285 s

30to60_spline-lineal_cheb.mp4


'Done'

In [38]:
interpolador = "spline-lineal"
old_fps = 15
dst_fps = 30
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 3587.716891181997 s
	Transcurrido lapso: 10.02043315300034 s
	Restante: 10.02043315300034 s

15to30_spline-lineal_equi.mp4


'Done'

In [39]:
interpolador = "spline-lineal"
old_fps = 15
dst_fps = 30
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 628.3362317680003 s
	Transcurrido lapso: 1.5444463359999645 s
	Restante: 1.5444463359999645 s

15to30_spline-lineal_cheb.mp4


'Done'

In [40]:
interpolador = "spline-lineal"
old_fps = 5
dst_fps = 15
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 556.8267403350001 s
	Transcurrido lapso: 1.566851969997515 s
	Restante: 1.566851969997515 s

5to15_spline-lineal_equi.mp4


'Done'

In [41]:
interpolador = "spline-lineal"
old_fps = 5
dst_fps = 15
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 122.24525107400041 s
	Transcurrido lapso: 0.34042970899827196 s
	Restante: 0.34042970899827196 s

5to15_spline-lineal_cheb.mp4


'Done'

In [42]:
interpolador = "spline-lineal"
old_fps = 5
dst_fps = 60
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 2218.200347676 s
	Transcurrido lapso: 6.148735802998999 s
	Restante: 6.148735802998999 s

5to60_spline-lineal_equi.mp4


'Done'

In [ ]:
interpolador = "spline-lineal"
old_fps = 5
dst_fps = 60
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 467.133012590999 s
	Transcurrido lapso: 1.3266752029994677 s
	Restante: 1.3266752029994677 s

5to60_spline-lineal_cheb.mp4


'Done'

baricentrica

In [ ]:
interpolador = "baricentrica"
old_fps = 30
dst_fps = 60
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 363.66579735100095 s
	Transcurrido lapso: 1.0212487280005007 s
	Restante: 1.0212487280005007 s

30to60_baricentrica_equi.mp4


'Done'

In [ ]:
interpolador = "baricentrica"
old_fps = 30
dst_fps = 60
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 353.54291033900154 s
	Transcurrido lapso: 0.9921859690002748 s
	Restante: 0.9921859690002748 s

30to60_baricentrica_cheb.mp4


'Done'

In [ ]:
interpolador = "baricentrica"
old_fps = 15
dst_fps = 30
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 187.598409025999 s
	Transcurrido lapso: 0.5088453919997846 s
	Restante: 0.5088453919997846 s

15to30_baricentrica_equi.mp4


'Done'

In [ ]:
interpolador = "baricentrica"
old_fps = 15
dst_fps = 30
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 186.140898362999 s
	Transcurrido lapso: 0.530511345998093 s
	Restante: 0.530511345998093 s

15to30_baricentrica_cheb.mp4


'Done'

In [ ]:
interpolador = "baricentrica"
old_fps = 5
dst_fps = 15
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 59.86372435700105 s
	Transcurrido lapso: 0.16630349700062652 s
	Restante: 0.16630349700062652 s

5to15_baricentrica_equi.mp4


'Done'

In [ ]:
interpolador = "baricentrica"
old_fps = 5
dst_fps = 15
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 59.46706873899893 s
	Transcurrido lapso: 0.16567331500118598 s
	Restante: 0.16567331500118598 s

5to15_baricentrica_cheb.mp4


'Done'

In [ ]:
interpolador = "baricentrica"
old_fps = 5
dst_fps = 60
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 71.82889896299821 s
	Transcurrido lapso: 0.20508027100004256 s
	Restante: 0.20508027100004256 s

5to60_baricentrica_equi.mp4


'Done'

In [ ]:
interpolador = "baricentrica"
old_fps = 5
dst_fps = 60
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

357 / 358
	Transcurrido total: 70.05616927099982 s
	Transcurrido lapso: 0.19654392899974482 s
	Restante: 0.19654392899974482 s

5to60_baricentrica_cheb.mp4


'Done'

lagrange

In [ ]:
interpolador = "lagrange"
old_fps = 30
dst_fps = 60
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

67 / 358
	Transcurrido total: 33567.732559646 s
	Transcurrido lapso: 559.4520006300008 s
	Restante: 162800.53218333024 s



In [ ]:
interpolador = "lagrange"
old_fps = 30
dst_fps = 60
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

In [ ]:
interpolador = "lagrange"
old_fps = 15
dst_fps = 30
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

In [ ]:
interpolador = "lagrange"
old_fps = 15
dst_fps = 30
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

In [ ]:
interpolador = "lagrange"
old_fps = 5
dst_fps = 15
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

In [ ]:
interpolador = "lagrange"
old_fps = 5
dst_fps = 15
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

In [ ]:
interpolador = "lagrange"
old_fps = 5
dst_fps = 60
puntos = "equi"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

In [ ]:
interpolador = "lagrange"
old_fps = 5
dst_fps = 60
puntos = "cheb"
nombre = str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4"
print(nombre)
interpolate_video(nombre, old_fps, dst_fps, interpolador, puntos)

In [ ]:
#seconds = 5
#old_fps = 15
#dst_fps = 30
#interpolador = "baricentrica"
#puntos = "cheb"
##ejemplo = video_to_matrix("video_"+str(old_fps)+"fps.mp4", seconds)
#asdf = interpolate_frames(ejemplo, old_fps, dst_fps, seconds, interpolador, puntos)
#matrix_to_video(str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4", asdf, dst_fps, asdf.shape[1], asdf.shape[0])

In [ ]:
#matrix_to_video(str(old_fps)+"to"+str(dst_fps)+"_"+interpolador+"_"+puntos+".mp4", asdf, dst_fps, asdf.shape[1], asdf.shape[0])



Para simplificar el manejo del video en el *notebook*, se les facilita la función `video_to_matrix` que les permitirá abrir un video y transformarlo a una estructura de datos de *numpy*. El resultado será una *array* de dimensión $\text{height}\times\text{width}\times\text{Total de frames}$, donde *height* corresponde a la altura (en pixeles) del video y *width* al ancho del video.

Además, el siguiente código sirve para visualizar un video dentro del mismo *notebook*.

In [ ]:
"""HTML('\
<div align="middle">\
<video width="80%" controls>\
      <source src="./{0}" type="video/mp4">\
</video></div>'.format('5to15_baricentrica_cheb'))"""

## Sección 2 (15 puntos). Visualización del interpolador y del error, a través del tiempo

1. Utilizando el *widget* adjunto, visualice los resultados generados por los interpoladores, su error y comente al respecto. ¿Qué método le parece que tiene un mejor desempeño?

**Nota**: usted debe modificar el *widget* en las secciones necesarias para que lea correctamente los videos generados por usted, acorde a los parámetros que se pueden variar. Puede modificar secciones del *widget* para que se adapte a sus códigos.

In [ ]:
# Widget to plot the interpolation function
def plot_interpolation(method, points_dist, i, j, fps_increase_type):
    
    # Elegimos el video a trabajar
    if fps_increase_type == "5->15":
        frames = video_to_matrix('video_15fps.mp4', 5)
        interpolated_video_name = 'video_15fps.mp4' # Acá va el video de 15 fps que generó a partir del de 5 fps
    elif fps_increase_type == "5->60":
        frames = video_to_matrix('video_60fps.mp4', 5)
        interpolated_video_name = 'video_60fps.mp4' # Acá va el video de 60 fps que generó a partir del de 5 fps
    elif fps_increase_type == "15->30":
        frames = video_to_matrix('video_30fps.mp4', 5)
        interpolated_video_name = 'video_30fps.mp4' # Acá va el video de 30 fps que generó a partir del de 15 fps
    else:
        frames = video_to_matrix('video_60fps.mp4', 5)
        interpolated_video_name = 'video_60fps.mp4' # Acá va el video de 60 fps que generó a partir del de 30 fps
        
    # Un punto por frame
    x = np.arange(0, frames.shape[2], 1)
    
    # Usar el interpolador acá (Matrix aleatoria es simplemente como un ejemplo para introducir ruido)
    interp_frames = video_to_matrix(interpolated_video_name, 5) + np.random.rand(frames.shape[0], frames.shape[1], frames.shape[2])
    
    plt.figure(figsize=(15, 14))
    
    # Grafico del interpolador y los pixeles reales
    plt.subplot(211)
    plt.plot(x, interp_frames[i,j,:], 'rx-', label='Video interpolados')
    plt.plot(x, frames[i,j,:], 'bd', label='Video esperado')
    plt.ylim([0, 256])
    plt.title('Comparación entre los pixeles interpolados y los deseados')
    plt.xlabel('Frame')
    plt.ylabel('Valor pixel')
    plt.grid(True)
    
    # Grafico del error
    plt.subplot(212)
    error_matrix = frames - interp_frames
    plt.plot(x, error_matrix[i,j,:], 'sk-', label='Error de interpolación')
    mean_error = np.zeros(error_matrix.shape[2])
    sigma = mean_error.copy()
    for k in range(error_matrix.shape[2]):
        mean_error[k] = error_matrix[:,:,k].mean()
        sigma[k] = error_matrix[:,:,k].std()
    plt.plot(x, mean_error, '.-', label='Error medio de interpolación')
    plt.plot(x, mean_error + sigma, label='Error medio + desviación estándar')
    plt.plot(x, mean_error - sigma, label='Error medio - desviación estándar')
    plt.title('Error de interpolación')
    plt.xlabel('Frame')
    plt.ylabel('Error')
    plt.grid(True)
    plt.legend(loc='best')
    
    plt.show()
"""
interact(
    plot_interpolation, 
    method=["spline-cubica","spline-lineal","lagrange","baricentrica"],
    points_dist=["equi","cheb"],
    j=(0, 511),
    i=(0, 269),
    fps_increase_type=["5->60", "5->15", "15->30", "30->60"]
);
"""

## Sección 3 (25 puntos). Comparación de Videos

1. Utilizando los videos generados en la pregunta anterior, compare visualmente los videos generados con su versión de "origen" (video original al que se le agregaron nuevos *frames*) y con la versión "esperada" del video; por ejemplo, para la interpolación de 15fps a 30fps, el video de origen sería _video_\__15fps.mp4_, y el video esperado _video_\__30fps.mp4_. Comente al respecto. ¿Se realiza correctamente la interpolación? ¿Qué es lo que está haciendo visualmente la interpolación en los videos?

    Para ayudar a la visualización de los videos, puede hacer uso de la función _concat_\__2_\__videos (src_\__name, dst_\__name, seconds)_, la cual permite concatenar o "unir" dos videos con el mismo framerate para realizar una comparativa en un nuevo video generado.

    _NOTA: Debe hacer un preprocesamiento sobre el video de origen para compararlo con la interpolación, pues la función solo permite unir 2 videos de igual framerate. Una solución simple es repetir cada frame la cantidad necesaria para completar el framerate deseado_ <br/><br/>

2. Comente respecto a los tiempos de cómputo obtenidos. ¿Existe alguna correlación entre la calidad del video generado y los tiempos de cómputo?

3. Comente respecto al efecto de la distribución de puntos. ¿Genera alguna diferencia la distribución de puntos equiespaciada respecto a los puntos de Chebyshev?

In [ ]:
'''
src_name: name/path of first video
dst_name: name/path of second video
seconds : seconds of video to consider.

returns: None
'''

def concat_2_videos(src_name, dst_name, seconds):
    stream = cv2.VideoCapture(src_name)
    src_frames = video_to_matrix(src_name, seconds)
    print("FPS VIDEO "+src_name+": ",int(src_frames.shape[2]/seconds))
    
    stream_2 = cv2.VideoCapture(dst_name)  
    dst_frames = video_to_matrix(dst_name, seconds)
    print("FPS VIDEO "+dst_name+": ",int(dst_frames.shape[2]/seconds))

    print(src_frames == dst_frames)
    
    # Close original videos
    stream.release()
    stream_2.release()
    
    if dst_frames.shape[2] != src_frames.shape[2]:
        print("No es posible concatenar dos videos de distinto fps")
        return
    
    fps = int(dst_frames.shape[2]/seconds)
    
    #concatenate both videos
    conc_frames = np.concatenate((src_frames, dst_frames), axis=1)
    
    height = conc_frames.shape[0]
    width = conc_frames.shape[1]
    
    #saving comparison on a new video
    print("comparison_"+src_name+"_vs_"+dst_name+".mp4")
    out = cv2.VideoWriter("comparison_"+src_name+"_vs_"+dst_name+".mp4",cv2.VideoWriter_fourcc('m','p','4','v'), fps, (width,height))
   
    for i in range(conc_frames.shape[2]):
        frame = np.uint8(conc_frames[:,:,i])
        out.write(np.dstack([frame, frame, frame]))
    
    out.release()
    cv2.destroyAllWindows() 

In [ ]:
#concat_2_videos("5to15_baricentrica_cheb_old.mp4", "5to15_baricentrica_cheb_sp.mp4", 5)

## Sección 4 (10 puntos): Conclusión

Considerando todo el trabajo anterior, concluya al respecto.

1. _¿En palabras simples, qué es lo que realiza el interpolador entre cada par de *frames*?. **Hint: Un análisis frame a frame puede ayudar a responder esto.**_

2. _¿De qué manera se pueden mejorar los métodos estudiados para aumentar el framerate de un video?. **Hint: ¿Es adecuado considerar el conjunto de pixeles como independientes entre sí?**_

# Referencias
[1] Stock footage provided by Videvo, downloaded from https://www.videvo.net

[2] Barycentric Lagrange Interpolation, https://people.maths.ox.ac.uk/trefethen/barycentric.pdf